In [2]:
import pandas as pd
import numpy as np
import pickle
import re
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [346]:
# import glob

# print(glob.glob('*.csv'))

# final_df = pd.DataFrame()
# for i in ['0_9999k_data.csv','10k_19k_data.csv', '20k_29k_data.csv']:
#     one_part = pd.read_csv(i)
#     final_df = final_df.append(one_part)

# final_df.to_pickle("27k_data.pkl")
# final_df.to_csv("27k_data.csv", index=False)

In [3]:
final_df = pd.read_csv('27k_data.zip')
final_df = final_df.rename(columns = lambda x : str(x).replace(':',''))
final_df = final_df.rename(columns = lambda x : str(x).replace(" ",'_'))
final_df = final_df.apply(lambda x: x.replace(np.NaN,''))
final_df = final_df.apply(lambda x: x.astype(str).str.lower())
final_df.shape

(26811, 18)

In [382]:
final_df.drop_duplicates('movie_name')

,Rating,Genre,Directed_By,Written_By,In_Theaters,On_Disc/Streaming,Runtime,Studio,movie_name,critic_consensus,critic_score,audience_score,top_5_celeb_names,critic_line,critic_url,audience_review,quotes,Box_Office
0,pg-13 (for sequences of violence),"action & adventure, drama, kids & family",niki caro,"rick jaffa, amanda silver, elizabeth martin, l...","aug 21, 2020 wide","sep 4, 2020",115 minutes,disney+,mulan,it could have told its classic story with grea...,75%,51%,"yifei-liu, donnie_yen, jason_scott_lee, yoson_...","i'm as athletic as a bed sock but, for so many...",https://www.standard.co.uk/go/london/film/mula...,the new mulan looked like something i've been ...,,
1,pg-13 (for some language),"action & adventure, comedy, science fiction & ...",dean parisot,"chris matheson, ed solomon","aug 28, 2020 wide","aug 28, 2020",,orion pictures,bill & ted face the music,"as wholesomely goofy as its heroes, bill and t...",81%,75%,"keanu_reeves, alex_winter, samara_weaving, bri...","for fans, the pair's all-round lameness has al...",https://www.smh.com.au/culture/movies/bill-and...,bill & ted face the music is one of those rare...,,
2,nr,documentary,gustavo sanchez,gustavo sanchez,,"sep 1, 2020",75 minutes,media luna films,i hate new york,,100%,,"bibbe_hansen, amanda_lepore, chloe_dzubilo, so...",,,,,
3,"r (for drug content, language throughout inclu...","action & adventure, comedy, horror",ninian doff,ninian doff,,"aug 28, 2020",86 minutes,amazon studios,get duked!,the rare comedy thriller that's funny as well ...,86%,62%,"samuel_bottomley, rian_gordon, lewis_gribben, ...",doff announces himself as a talent to be recko...,"https://www.reelviews.net/reelviews/get-duked,...",,,
4,nr,"drama, gay & lesbian",flavio alves,"john rotondo, flavio alves","aug 28, 2020 limited","sep 8, 2020",88 minutes,uncork'd entertainment,the garden left behind,the garden left behind draws on one immigrant'...,92%,94%,"michael_madsen, edward_asner, carlie_guevara, ...",,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26806,nr,"classics, drama, romance",delmer daves,delmer daves,"oct 22, 1959 wide","feb 6, 2007",130 minutes,warner home video,a summer place,,83%,70%,"dorothy_mcguire, richard_egan, arthur_kennedy,...","overlong and overwrought...march 19, 2007| rat...","http://www.reelfilm.com/warnrom1.htm#summer, h...",if you've seen grease you've heard that song a...,,
26807,nr,"art house & international, drama, romance",satyajit ray,satyajit ray,"jan 1, 1964 limited","feb 6, 2007",117 minutes,amanda films ltd,charulata,,93%,91%,"soumitra_chatterji, madhabi_mukherjee, shailen...",,,this beautiful film from director satyajit ray...,,
26808,r,"classics, drama",elia kazan,elia kazan,"nov 18, 1969 wide","jan 30, 2007",125 minutes,warner home video,the arrangement,,8%,65%,"kirk_douglas_3, deborah_kerr, faye_dunaway, ri...",,,flat searching for meaning drama with high qua...,,
26809,nr (adult situations),"classics, documentary, drama",mervyn leroy,"paul osborn, paul h. rameau","dec 16, 1943 limited","jan 30, 2007",124 minutes,warner bros. pictures,madame curie,,86%,66%,"greer_garson, walter_pidgeon, robert_walker, m...",,,pidgeon and garson get together often in films...,,


In [349]:
def stop_words_removal(only_text):
    only_text = only_text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    return only_text

In [350]:
# rating, genre, directed by, release date ( In_Theaters), Runtime, Studio, Movie_name, top_5_celeb_name, rating_words

In [351]:
final_df['Rating_words'] = final_df['Rating'].str.split("\s+\(").str[1]
final_df['Rating_words'] = final_df.Rating_words.astype(str)
final_df['Rating_words'] = final_df.Rating_words.str.replace('[^a-z \n]',"")
final_df['Rating_words'] = stop_words_removal(final_df.Rating_words)

final_df['Rating'] = final_df['Rating'].str.split("\s+\(").str[0]
final_df['Rating'] = final_df.Rating.replace("nr",'')

In [352]:
final_df['Genre'] = stop_words_removal(final_df.Genre)
final_df['Genre'] = final_df.Genre.str.replace('[^a-z \n]',"")

In [353]:
final_df['Directed_By'] = final_df.Directed_By.str.replace('[^a-z \n]',"")
final_df['Directed_By_1'] = final_df['Directed_By'].str.split(',').str[0]
final_df['Directed_By_2'] = final_df['Directed_By'].str.split(',').str[1]

In [362]:
final_df['release_year'] = final_df.In_Theaters.str.split(' ').str[2]
final_df['release_year'] = final_df.release_year.str.replace('[^0-9 \n]',"")

final_df['release_year'] = final_df.release_year.fillna(0)
final_df['release_year'] = final_df.release_year.astype(int)


final_df.loc[(final_df['release_year'] >=1920) & (final_df['release_year'] <1950), 'release_year_brackets'] ='black_white'
final_df.loc[(final_df['release_year'] >=1950) & (final_df['release_year'] <=1980), 'release_year_brackets'] ='pretty_old'
final_df.loc[(final_df['release_year'] >=1980) & (final_df['release_year'] <=1990), 'release_year_brackets'] ='year_80_90'
final_df.loc[(final_df['release_year'] >=1990) & (final_df['release_year'] <=2000), 'release_year_brackets'] ='year_90_2000'
final_df.loc[(final_df['release_year'] >=2000) & (final_df['release_year'] <=2010), 'release_year_brackets'] ='year_2000_2010'
final_df.loc[(final_df['release_year'] >=2010) & (final_df['release_year'] <=2020), 'release_year_brackets'] ='year_2010_2020'

final_df.loc[(final_df.release_year == 0), 'release_year_brackets'] = ''

In [360]:
final_df['Runtime'] = final_df.Runtime.str.split(" ").str[0]
final_df['Runtime'] = final_df.Runtime.replace("",np.NaN)
final_df['Runtime'] = final_df['Runtime'].fillna(0)
final_df['Runtime'] = final_df['Runtime'].astype(int)

In [370]:
final_df.loc[(final_df['Runtime'] > 10) & (final_df['Runtime'] < 90), 'runtime_brackets'] = 'short_till90'
final_df.loc[(final_df['Runtime'] >= 90) & (final_df['Runtime'] < 120), 'runtime_brackets'] = 'conventional_90_120'
final_df.loc[(final_df['Runtime'] >= 120) & (final_df['Runtime'] < 180), 'runtime_brackets'] = 'lengthy_120_180'
final_df.loc[(final_df['Runtime'] >= 180) & (final_df['Runtime'] < 800), 'runtime_brackets'] = 'super_lengthy_180_800'

final_df.loc[(final_df.runtime_brackets == 0), 'runtime_brackets'] = ''

In [379]:
final_df.shape

(26811, 24)

In [ ]:
## final_df.Runtime.value_counts()

In [378]:
oo = pd.DataFrame(final_df.Studio.value_counts())
oo[oo.Studio >100]

,Studio
netflix,1064
ifc films,1027
,957
magnolia pictures,610
warner bros. pictures,596
universal pictures,564
20th century fox,549
sony pictures classics,459
paramount pictures,398
gravitas ventures,372


In [13]:
movie_freq = pd.DataFrame(final_df.movie_nam.value_counts())
movie_freq[movie_freq.movie_nam > 1]

,movie_nam
Shirley,5
Road to Paloma,3
Capital,3
First Comes Love,3
Child Of God,3
...,...
Saint,2
You Kill Me,2
Heart of the Beholder,2
School for Scoundrels,2


In [14]:
final_df = final_df.drop_duplicates(subset='movie_nam')
final_df.shape

(9605, 18)

In [19]:
final_df.to_csv("asdf.csv")

In [15]:
final_df.critic_scor.value_counts()

100%    570
50%     233
88%     225
80%     218
83%     212
       ... 
5%       24
6%       22
3%       10
2%        6
1%        3
Name: critic_scor, Length: 101, dtype: int64

In [16]:
final_df.audience_scor.value_counts()

67%    179
71%    175
72%    166
50%    162
73%    159
      ... 
4%       3
5%       3
6%       2
7%       2
3%       1
Name: audience_scor, Length: 99, dtype: int64

In [17]:
final_df.top_5_celeb_name

0        yifei-liu, donnie_yen, jason_scott_lee, yoson_...
1        keanu_reeves, alex_winter, samara_weaving, bri...
2        bibbe_hansen, amanda_lepore, chloe_dzubilo, so...
3        samuel_bottomley, rian_gordon, lewis_gribben, ...
4        michael_madsen, edward_asner, carlie_guevara, ...
                               ...                        
26806    dorothy_mcguire, richard_egan, arthur_kennedy,...
26807    soumitra_chatterji, madhabi_mukherjee, shailen...
26808    kirk_douglas_3, deborah_kerr, faye_dunaway, ri...
26809    greer_garson, walter_pidgeon, robert_walker, m...
26810    1014038-george_segal, susan_anspach, kris_kris...
Name: top_5_celeb_name, Length: 9605, dtype: object

In [18]:
final_df.isna().sum()

Rating                  0
Genre                  21
Directed_By           125
Written_By           1006
In_Theaters           917
On_Disc/Streaming       2
Runtime               178
Studio                336
movie_nam               0
critic_consensu      4598
critic_scor             0
audience_scor         325
top_5_celeb_name      288
critic_lin           3274
critic_ur            3274
audience_revie       2831
quote                7383
Box_Office           5870
dtype: int64